# 损失函数

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/advanced/network/mindspore_loss.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/advanced/network/mindspore_loss.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/advanced/network/loss.ipynb)

损失函数，亦称目标函数，用于衡量预测值与真实值差异的程度。

在深度学习中，模型训练就是通过不断迭代来缩小损失函数值的过程。因此，在模型训练过程中损失函数的选择非常重要，一个好的损失函数能有效提升模型的性能。

`mindspore.nn`模块中提供了许多[通用损失函数](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore.nn.html#损失函数)，但这些通用损失函数无法满足所有需求，很多情况需要用户自定义所需的损失函数。因此，本教程介绍如何自定义损失函数。

![lossfun.png](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/tutorials/source_zh_cn/advanced/network/images/loss_function.png)

## 内置损失函数

首先介绍`mindspore.nn`模块中内置的[损失函数](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore.nn.html#损失函数)。

下例以`nn.L1Loss`为例，计算预测值和目标值之间的平均绝对误差：

$$\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad \text{with } l_n = \left| x_n - y_n \right|$$

其中N为数据集中的batch size大小。

$$\ell(x, y) =
        \begin{cases}
            \operatorname{mean}(L), & \text{if reduction} = \text{'mean';}\\
            \operatorname{sum}(L),  & \text{if reduction} = \text{'sum'.}
        \end{cases}$$

`nn.L1Loss`中的参数`reduction`取值可为`mean`，`sum`，或`none`。若`reduction` 为`mean`或`sum`，则输出一个经过均值或求和后的标量Tensor（降维）；若`reduction`为`none`，则所输出Tensor的shape为广播后的shape。

In [1]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor

# 输出loss均值
loss = nn.L1Loss()
# 输出loss和
loss_sum = nn.L1Loss(reduction='sum')
# 输出loss原值
loss_none = nn.L1Loss(reduction='none')

input_data = Tensor(np.array([[1, 2, 3], [2, 3, 4]]).astype(np.float32))
target_data = Tensor(np.array([[0, 2, 5], [3, 1, 1]]).astype(np.float32))

print("loss:", loss(input_data, target_data))
print("loss_sum:", loss_sum(input_data, target_data))
print("loss_none:\n", loss_none(input_data, target_data))

loss: 1.5
loss_sum: 9.0
loss_none:
 [[1. 0. 2.]
 [1. 2. 3.]]


## 自定义损失函数

自定义损失函数的方法有两种：一是基于`nn.Cell`来定义损失函数；二是`nn.LossBase`来定义损失函数。`nn.LossBase`继承自`nn.Cell`，额外提供了`get_loss`方法，利用`reduction`参数对损失值求和或求均值，输出一个标量。

下面将分别使用继承`Cell`和继承`LossBase`的方法，来定义平均绝对误差损失函数(Mean Absolute Error，MAE)，MAE算法的公式如下所示：

$$ loss= \frac{1}{m}\sum_{i=1}^m\lvert y_i-f(x_i) \rvert$$

上式中$f(x)$为预测值，$y$为样本真实值，$loss$为预测值与真实值之间距离的平均值。

### 基于`nn.Cell`构造损失函数

`nn.Cell`是MindSpore的基类，不但可用于构建网络，还可用于定义损失函数。使用`nn.Cell`定义损失函数的过程与定义一个普通的网络相似，差别在于，其执行逻辑部分要计算的是前向网络输出与真实值之间的误差。

下面展示怎样基于`nn.Cell`自定义损失函数`MAELoss`：

In [2]:
import mindspore.ops as ops

class MAELoss(nn.Cell):
    """自定义损失函数MAELoss"""

    def __init__(self):
        """初始化"""
        super(MAELoss, self).__init__()
        self.abs = ops.Abs()
        self.reduce_mean = ops.ReduceMean()

    def construct(self, base, target):
        """调用算子"""
        x = self.abs(base - target)
        return self.reduce_mean(x)

loss = MAELoss()

input_data = Tensor(np.array([0.1, 0.2, 0.3]).astype(np.float32))  # 生成预测值
target_data = Tensor(np.array([0.1, 0.2, 0.2]).astype(np.float32)) # 生成真实值

output = loss(input_data, target_data)
print(output)

0.033333335


### 基于`nn.LossBase`构造损失函数

基于[nn.LossBase](https://www.mindspore.cn/docs/zh-CN/master/api_python/nn/mindspore.nn.LossBase.html#mindspore.nn.LossBase)构造损失函数`MAELoss`与基于`nn.Cell`构造损失函数的过程类似，都要重写`__init__`方法和`construct`方法。

`nn.LossBase`可使用方法`get_loss`将`reduction`应用于损失计算。

In [4]:
class MAELoss(nn.LossBase):
    """自定义损失函数MAELoss"""

    def __init__(self, reduction="mean"):
        """初始化并求loss均值"""
        super(MAELoss, self).__init__(reduction)
        self.abs = ops.Abs()  # 求绝对值算子

    def construct(self, base, target):
        x = self.abs(base - target)
        return self.get_loss(x)  # 返回loss均值

loss = MAELoss()

input_data = Tensor(np.array([0.1, 0.2, 0.3]).astype(np.float32))  # 生成预测值
target_data = Tensor(np.array([0.1, 0.2, 0.2]).astype(np.float32))  # 生成真实值

output = loss(input_data, target_data)
print(output)

0.033333335


## 损失函数与模型训练

损失函数`MAELoss`自定义完成后，可使用MindSpore的接口[Model](https://www.mindspore.cn/docs/zh-CN/master/api_python/mindspore/mindspore.Model.html#mindspore.Model)中`train`接口进行模型训练，构造`Model`时需传入前向网络、损失函数和优化器，`Model`会在内部将它们关联起来，生成一个可用于训练的网络模型。

在`Model`中，前向网络和损失函数通过[nn.WithLossCell](https://www.mindspore.cn/docs/zh-CN/master/api_python/nn/mindspore.nn.WithLossCell.html#mindspore.nn.WithLossCell)关联起来，`nn.WithLossCell`支持两个输入，分别为`data`和`label`。

In [5]:
from mindspore import Model
from mindspore import dataset as ds
from mindspore.common.initializer import Normal
from mindvision.engine.callback import LossMonitor

def get_data(num, w=2.0, b=3.0):
    """生成数据及对应标签"""
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise = np.random.normal(0, 1)
        y = x * w + b + noise
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)

def create_dataset(num_data, batch_size=16):
    """加载数据集"""
    dataset = ds.GeneratorDataset(list(get_data(num_data)), column_names=['data', 'label'])
    dataset = dataset.batch(batch_size)
    return dataset

class LinearNet(nn.Cell):
    """定义线性回归网络"""
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)

ds_train = create_dataset(num_data=160)
net = LinearNet()
loss = MAELoss()
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

# 使用model接口将网络、损失函数和优化器关联起来
model = Model(net, loss, opt)
model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor(0.005)])

Epoch:[  0/  1], step:[    1/   10], loss:[9.169/9.169], time:365.966 ms, lr:0.00500
Epoch:[  0/  1], step:[    2/   10], loss:[5.861/7.515], time:0.806 ms, lr:0.00500
Epoch:[  0/  1], step:[    3/   10], loss:[8.759/7.930], time:0.768 ms, lr:0.00500
Epoch:[  0/  1], step:[    4/   10], loss:[9.503/8.323], time:1.080 ms, lr:0.00500
Epoch:[  0/  1], step:[    5/   10], loss:[8.541/8.367], time:0.762 ms, lr:0.00500
Epoch:[  0/  1], step:[    6/   10], loss:[9.158/8.499], time:0.707 ms, lr:0.00500
Epoch:[  0/  1], step:[    7/   10], loss:[9.168/8.594], time:0.900 ms, lr:0.00500
Epoch:[  0/  1], step:[    8/   10], loss:[6.828/8.373], time:1.184 ms, lr:0.00500
Epoch:[  0/  1], step:[    9/   10], loss:[7.149/8.237], time:0.962 ms, lr:0.00500
Epoch:[  0/  1], step:[   10/   10], loss:[6.342/8.048], time:1.273 ms, lr:0.00500
Epoch time: 390.358 ms, per step time: 39.036 ms, avg loss: 8.048


## 多标签损失函数与模型训练

上述定义了一个简单的平均绝对误差损失函数`MAELoss`，但许多深度学习应用的数据集较复杂，如目标检测网络Faster R-CNN的数据中就包含多个标签，而非简单的一条数据对应一个标签，这时损失函数的定义和使用略有不同。

本节介绍在多标签数据集场景下，如何定义多标签损失函数（Multi label loss function），并使用Model进行模型训练。

### 多标签数据集

下例通过`get_multilabel_data`函数拟合两组线性数据$y1$和$y2$，拟合的目标函数为：

$$f(x)=2x+3$$

最终数据集应随机分布于函数周边，这里按以下公式的方式生成，其中`noise`为服从标准正态分布的随机值。`get_multilabel_data`函数返回数据$x$、$y1$和$y2$：

$$f(x)=2x+3+noise$$

通过`create_multilabel_dataset`生成多标签数据集，并将`GeneratorDataset`中的`column_names`参数设置为['data', 'label1', 'label2']，最终返回的数据集格式为一条数据`data`对应两个标签`label1`和`label2`。

In [6]:
import numpy as np
from mindspore import dataset as ds

def get_multilabel_data(num, w=2.0, b=3.0):
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise1 = np.random.normal(0, 1)
        noise2 = np.random.normal(-1, 1)
        y1 = x * w + b + noise1
        y2 = x * w + b + noise2
        yield np.array([x]).astype(np.float32), np.array([y1]).astype(np.float32), np.array([y2]).astype(np.float32)

def create_multilabel_dataset(num_data, batch_size=16):
    dataset = ds.GeneratorDataset(list(get_multilabel_data(num_data)), column_names=['data', 'label1', 'label2'])
    dataset = dataset.batch(batch_size)  # 每个batch有16个数据
    return dataset

### 多标签损失函数

针对上一步创建的多标签数据集，定义多标签损失函数`MAELossForMultiLabel`。

$$ loss1= \frac{1}{m}\sum_{i=1}^m\lvert y1_i-f(x_i) \rvert$$

$$ loss2= \frac{1}{m}\sum_{i=1}^m\lvert y2_i-f(x_i) \rvert$$

$$ loss = \frac{(loss1 + loss2)}{2}$$

上式中，$f(x)$ 为样例标签的预测值，$y1$ 和 $y2$ 为样例标签的真实值，$loss1$ 为预测值与真实值 $y1$ 之间距离的平均值，$loss2$ 为预测值与真实值 $y2$ 之间距离的平均值 ，$loss$ 为损失值 $loss1$ 与损失值 $loss2$ 平均值。

在`MAELossForMultiLabel`中的`construct`方法的输入有三个，预测值`base`，真实值`target1`和`target2`，在`construct`中分别计算预测值与真实值`target1`、预测值与真实值`target2`之间的误差，将两误差取平均后作为最终的损失函数值.

示例代码如下：

In [7]:
class MAELossForMultiLabel(nn.LossBase):
    def __init__(self, reduction="mean"):
        super(MAELossForMultiLabel, self).__init__(reduction)
        self.abs = ops.Abs()

    def construct(self, base, target1, target2):
        x1 = self.abs(base - target1)
        x2 = self.abs(base - target2)
        return (self.get_loss(x1) + self.get_loss(x2))/2

### 多标签模型训练

使用`Model`关联指定的前向网络、损失函数和优化器时，因`Model`内默认使用的`nn.WithLossCell`只接受两个输入：`data`和`label`，故不适用于多标签场景。

在多标签场景下，若想使用`Model`进行模型训练，则需事先把前向网络与多标签损失函数关联起来，即自定义损失网络。

- 定义损失网络

下例展示了如何定义损失网络`CustomWithLossCell`，其中`__init__`方法的两个参数`backbone`和`loss_fn`分别表示前向网络和损失函数，`construct`方法的输入分别为样例输入`data`和样例真实标签`label1`、`label2`，将样例输入`data`传给前向网络`backbone`，将预测值和两标签值传给损失函数`loss_fn`。

In [8]:
class CustomWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn):
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn

    def construct(self, data, label1, label2):
        output = self._backbone(data)
        return self._loss_fn(output, label1, label2)

- 定义网络模型并训练

使用Model连接前向网络、多标签损失函数和优化器时，`Model`的网络`network`指定为自定义的损失网络`loss_net`，损失函数`loss_fn`不指定，优化器仍使用`Momentum`。

在未指定`loss_fn`时，`Model`会默认`network`内部已实现损失函数的逻辑，不再在内部使用`nn.WithLossCell`关联前向网络和损失函数。

In [9]:
ds_train = create_multilabel_dataset(num_data=160)
net = LinearNet()

# 定义多标签损失函数
loss = MAELossForMultiLabel()

# 定义损失网络，连接前向网络和多标签损失函数
loss_net = CustomWithLossCell(net, loss)

# 定义优化器
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

# 定义Model，多标签场景下Model无需指定损失函数
model = Model(network=loss_net, optimizer=opt)

model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor(0.005)])

Epoch:[  0/  1], step:[    1/   10], loss:[10.329/10.329], time:290.788 ms, lr:0.00500
Epoch:[  0/  1], step:[    2/   10], loss:[10.134/10.231], time:0.813 ms, lr:0.00500
Epoch:[  0/  1], step:[    3/   10], loss:[9.862/10.108], time:2.410 ms, lr:0.00500
Epoch:[  0/  1], step:[    4/   10], loss:[11.182/10.377], time:1.154 ms, lr:0.00500
Epoch:[  0/  1], step:[    5/   10], loss:[8.571/10.015], time:1.137 ms, lr:0.00500
Epoch:[  0/  1], step:[    6/   10], loss:[7.763/9.640], time:0.928 ms, lr:0.00500
Epoch:[  0/  1], step:[    7/   10], loss:[7.542/9.340], time:1.001 ms, lr:0.00500
Epoch:[  0/  1], step:[    8/   10], loss:[8.644/9.253], time:1.156 ms, lr:0.00500
Epoch:[  0/  1], step:[    9/   10], loss:[5.815/8.871], time:1.908 ms, lr:0.00500
Epoch:[  0/  1], step:[   10/   10], loss:[5.086/8.493], time:1.575 ms, lr:0.00500
Epoch time: 323.467 ms, per step time: 32.347 ms, avg loss: 8.493


本章节简单讲解了多标签数据集场景下，如何定义损失函数并使用Model进行模型训练。在很多其他场景中，也可采用此类方法进行模型训练。